In [1]:
import pandas as pd
import numpy as np
import pdb
from docx import Document
from docx.shared import Cm
from collections import defaultdict

In [2]:
MERSummary = pd.read_excel('MERWeek20.xlsx')
EMPS = pd.read_excel('EMPSWeek20.xlsx')
PDTD = pd.read_excel('PlantDowntimeDetailWk20.xlsx')

In [3]:
'''fleet desc import'''
fleetdesc = pd.read_json('fleetdesc.json')
fleetdesc.rename(columns={'Unit no':'Unit'},inplace=True)

In [4]:
'''
MER Processing

'''

MERSummary.drop(MERSummary.columns[[0,1]],axis=1,inplace=True)
MERSummary.dropna(inplace=True)
MERSummary.reset_index(drop=True,inplace=True)
MERSummary.columns = MERSummary.iloc[0]
MERSummary.drop(0,inplace=True)
indexNames = MERSummary[(MERSummary['Unit']=='OEM Totals')|(MERSummary['Unit']=='Class Total')|(MERSummary['Unit']=='Report Total')].index
MERSummary.drop(indexNames, inplace = True)

In [5]:
'''
EMPS Processing

'''

EMPS.drop(EMPS.columns[[0]],axis=1,inplace=True)
EMPS.dropna(inplace=True)
EMPS.reset_index(drop=True,inplace=True)
EMPS.columns = EMPS.iloc[0]
EMPS.drop(0,inplace=True)
EMPS.rename(columns = {'Unit\nNumber':'Unit'},inplace=True)
indexNames2 = EMPS[(EMPS['Unit']=='OEM Totals')|(EMPS['Unit']=='Class Total')|(EMPS['Unit']=='Report Total')].index
EMPS.drop(indexNames2,inplace=True)

In [6]:
'''Processing PDTD'''
PDTD.dropna(thresh=3,inplace=True) #masih PR ya
PDTD.reset_index(drop=True,inplace=True)
PDTD.columns = PDTD.iloc[0]
PDTD.drop(0,inplace=True)
PDTD.rename(columns = {'Unit No.' : 'Unit'},inplace=True)
PDTD_complete = pd.merge(PDTD, fleetdesc, on='Unit', how='inner')

In [7]:
'''Combine MER & EMPS'''
Combined = pd.merge(MERSummary,EMPS, on='Unit', how='outer')

In [8]:
'''Combine fleet desc & combined'''
All_KPI = pd.merge(Combined,fleetdesc,on='Unit', how='outer')
All_KPI.drop(columns=['Check','Column7','Column8'],inplace=True)
All_KPI.dropna()

,Unit,PWT\n(00),SWT\n(01),IOD-On\n(04),IOD-Off\n(04),EOD-On\n(05),EOD-Off\n(05),PMD\n(08),UMD\n(09),NRT On\n(15),...,MTTR-F\nPrevious 28 Days,Mech\nAvail\nPrevious 28 Days,Planned\nMaint %\nPrevious 28 Days,Physical\nAvail\nPrevious 28 Days,Opportune\nEfficiency\n% Previous 28 Days,Plant\nDamage %\nPrevious 28 Days,unit desc,fleet desc,additional,Column6
0,VT2210,0.0,69.00001,0.0,81.66664,0.0,0.0,0.16667,11.16667,0.0,...,7.202381,0.798926,0.070521,0.798926,0.000548,0,ISU NPS75,Medium Truck,Manhaul,Support
1,VT2212,0.0,67.00002,5.00006,83.58326,0.0,0.0,0.0,5.91666,0.0,...,2.625,0.945856,0.361656,0.945856,0.008248,0,ISU NPS75,Medium Truck,Manhaul,Support
2,VT2213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168.0,0.0,...,1,0,0,0,0,0,ISU NPS75,Medium Truck,Manhaul,Support
3,VT2214,0.0,65.00004,0.0,89.16663,0.0,0.0,0.0,7.33333,0.0,...,7.266666,0.90637,0.070652,0.90637,0.000854,0,ISU NPS75,Medium Truck,Manhaul,Support
4,VT3155,0.0,48.99997,0.0,92.83341,0.0,0.0,0.0,20.16666,0.0,...,4.958333,0.918718,0.078616,0.918718,0.000375,0,IVE AD240 4x4 MANHAUL TRUCK,Medium Truck,Manhaul,Support
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,VT1030,0.0,97.99999,0.0,64.50001,0.0,0.0,0.0,0.0,0.0,...,4.75,0.981269,0,0.981269,0,0,VOL FM440,Medium Truck,Fuel Trailer,Support
443,VT1064,0.0,11.0,0.0,151.5,0.0,0.0,0.0,0.0,0.0,...,6.25,0.946968,0.643836,0.946968,0.003804,0,VOL FM440,Medium Truck,Water Trailer,Support
444,VT1261,0.0,35.99999,0.0,88.33334,0.0,0.0,0.0,40.16667,0.0,...,28.083335,0.925051,0.12532,0.925051,0.034127,0,VOL FM440,Medium Truck,Fuel Trailer,Support
445,VT1215,0.0,0.0,0.0,163.0,0.0,0.0,0.0,0.0,0.0,...,1,0.938168,0.234568,0.938168,0,0,VOL FMX330,Medium Truck,Service Truck,Support


In [9]:
'''Gather all required variables'''
PWT = All_KPI.groupby(['fleet desc'])['PWT\n(00)'].sum() #F
SWT = All_KPI.groupby(['fleet desc'])['SWT\n(01)'].sum() #G
IODOn = All_KPI.groupby(['fleet desc'])['IOD-On\n(04)'].sum() #H
IODOff = All_KPI.groupby(['fleet desc'])['IOD-Off\n(04)'].sum() #I
EODOn = All_KPI.groupby(['fleet desc'])['EOD-On\n(05)'].sum() #J
EODOff = All_KPI.groupby(['fleet desc'])['EOD-Off\n(05)'].sum() #K
PDAM = All_KPI.groupby(['fleet desc'])['PDAM\n08020\n09020'].sum() #T
PMD = All_KPI.groupby(['fleet desc'])['PMD\n(08)'].sum() #L
UMD = All_KPI.groupby(['fleet desc'])['UMD\n(09)'].sum() #M
EngOn = All_KPI.groupby(['fleet desc'])['Total\nEng On\n(SMU Hrs)'].sum() #V
nFail = All_KPI.groupby(['fleet desc'])['Number of\nFailures\nPeriod'].sum() #W
MTBF_period = All_KPI.groupby(['fleet desc'])['MTBF\nPeriod'].sum() #X
MTTRF_period = All_KPI.groupby(['fleet desc'])['MTTR-F\nPeriod'].sum() #Y

In [10]:
'''Do the math'''
MA = ((PMD+UMD+IODOn+IODOff+EODOn+EODOff+PDAM)/(PMD+UMD+IODOn+IODOff+EODOn+EODOff+PDAM+UMD-PDAM).replace({0:np.nan})).fillna(0)
MTBF =(EngOn/(nFail).replace({0:np.nan})).fillna(EngOn)
MTTR = ((MTTRF_period*MTBF_period)/(MTBF_period).replace({0:np.nan})).fillna(MTTRF_period*MTBF_period)

In [11]:
'''Combined all KPI'''
Anomali_db = pd.merge(MA.to_frame(),MTBF.to_frame(), on='fleet desc', how= 'outer')
complete_anomali = pd.merge(Anomali_db, MTTR.to_frame(), on='fleet desc', how= 'outer').round(4)
complete_anomali.rename(columns = {'0_x':'MA','0_y':'MTBF'},inplace=True)
complete_anomali.columns = ['MA','MTBF','MTTR']
'''round by 2 decimals'''
complete_anomali['MTBF'] = complete_anomali['MTBF'].apply(lambda x: round(x,2)) 
complete_anomali['MTTR'] = complete_anomali['MTTR'].apply(lambda x: round(x,2))
'''drop zeros'''
complete_KPI = complete_anomali.loc[~(complete_anomali==0).all(axis=1)]
complete_KPI

,MA,MTBF,MTTR
fleet desc,,,
CAT 18M,0.8727,144.65,41.00
CAT 745,0.9005,121.93,47.25
Cat 16M,0.8597,70.64,31.00
Cat 740,0.8218,51.27,192.97
Cat 777D Coal Truck,0.9636,96.84,79.38
Cat 777D OB Truck,0.9496,84.40,76.67
Cat 777E Coal Truck,0.9792,115.04,30.62
Cat 785C,0.9000,71.30,412.38
Cat 789C,1.0000,4.20,1.00


Processing MTBF

In [12]:
red_MTBF_this_week = complete_KPI['MTBF'][complete_KPI['MTBF']<100].dropna()
MTBF_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MTBF_this_week.index)]
red_MTBF_this_week

fleet desc
Cat 16M                    70.64
Cat 740                    51.27
Cat 777D Coal Truck        96.84
Cat 777D OB Truck          84.40
Cat 785C                   71.30
Cat 789C                    4.20
Cat 834                    61.12
Cat D10R/T                 79.94
Cat D8R                    85.29
Compactor                  30.50
Crane                      97.00
Hitachi EX2500             49.00
Hitachi EX2600             77.60
Hitachi EX3600             44.84
Hitachi EX5500             29.40
Hitachi ZX200              41.60
KOM D85ESS                 47.86
Komatsu D65P-12            94.80
Komatsu PC300              37.61
Komatsu PC400              70.00
Liebherr R984              53.76
Medium Truck               32.01
Mud Truck                  36.94
TJ RENT Komatsu D65P-12    32.90
TJ RENT Komatsu D85ESS     17.10
TJ Rental PC200            66.70
Trailer/Lowboy             41.00
Volvo A40 Thiess           48.60
Water Truck                48.35
Name: MTBF, dtype: float64

In [190]:
'''MTBF Report template'''
MTBF_report = []


for i in red_MTBF_this_week.index:
    comp_gp_issue = []
    
    '''
    filter downtime detail for only failure event 
    '''
    
    na = MTBF_df[(MTBF_df['Is Failure']=='Yes')&(MTBF_df['fleet desc'] ==i)][['Reason','System']].groupby(['Reason'],as_index=False).count().sort_values('System',ascending=False)['Reason']
    count_of_failure = MTBF_df[(MTBF_df['fleet desc'] ==i)&(MTBF_df['Is Failure']=='Yes')].count()['Is Failure']
    '''
    conditional statement for unit that has no failure
    '''
    if count_of_failure == 0:
        comp_gp_issue.append([])
    else:
        for u in na:    
            '''
            Create dataframe for system failure
            '''
            system_failure = MTBF_df[(MTBF_df['Is Failure']=='Yes')][['fleet desc','Reason','System','Part']].groupby(['fleet desc','Reason','System'],as_index=False).count()
            system_failure['MTBF_gp_repetitive'] = system_failure['System']+' '+system_failure['Part'].values.astype(str)+' times'
            
            '''
            Create dataframe for reason failure failure
            '''
            reason_failure = MTBF_df[(MTBF_df['fleet desc']==i) & (MTBF_df['Is Failure']=='Yes')]['Reason'].value_counts().reset_index()
            reason_failure['Reason_repetitive'] = reason_failure['index']+' '+reason_failure['Reason'].values.astype(str)+' times'

            general_comp_failure = []
            for e in reason_failure['index']:
                f = system_failure[(system_failure['fleet desc']==i)&(system_failure['Reason']==e)]['MTBF_gp_repetitive'].tolist()
                general_comp_failure.append(f)
            repetitive_issue = dict(zip(reason_failure['Reason_repetitive'],general_comp_failure))
        comp_gp_issue.append(repetitive_issue)
    '''
    report template per fleet model
    
    '''
    
    structured_MTBF = {
    'Fleet':i,
    'MTBF':red_MTBF_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'Total Hours Engine ON' : All_KPI.groupby('fleet desc').sum()['Total\nEngine On\nPeriod'][i],
    'Count of failures' : count_of_failure,
    'Failures by Comp Gp with repetitive issue' : comp_gp_issue 
    }
    
    MTBF_report.append(structured_MTBF)
    
MTBF_report

[{'Fleet': 'Cat 16M',
  'MTBF': 70.64,
  'Total Unit': 8,
  'Total Hours Engine ON': 494.49941,
  'Count of failures': 7,
  'Failures by Comp Gp with repetitive issue': [{'018 - Chassis/Frame/Cabin/Steer Axles 2 times': ['Steering Mechanical 2 times'],
    '017 - Hydraulics 2 times': ['Electrical 1 times',
     'Steering Hydraulic Delivery & Cooling 1 times'],
    '019 - LV Electrical 1 times': ['Power 1 times'],
    '012 - Torque Convertor/Transmission 1 times': ['Transmission 1 times'],
    '025 - Fire Suppression 1 times': ['Storage 1 times']}]},
 {'Fleet': 'Cat 740',
  'MTBF': 51.27,
  'Total Unit': 23,
  'Total Hours Engine ON': 1025.4996099999998,
  'Count of failures': 21,
  'Failures by Comp Gp with repetitive issue': [{'018 - Chassis/Frame/Cabin/Steer Axles 7 times': ['Cabin 6 times',
     'Chassis 1 times'],
    '011 - Engine 4 times': ['Cooling 1 times',
     'Electrical 1 times',
     'Intake/Exhaust 2 times'],
    '012 - Torque Convertor/Transmission 2 times': ['Torque Con

Processing MA

In [196]:
red_MA_this_week = complete_KPI['MA'][complete_KPI['MA']<0.9].dropna()
MA_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MA_this_week.index)]
MA_df['MA Anomali']= MA_df['Unit']+' '+MA_df['Reported Fault/Job Description']+' ('+MA_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"
red_MA_this_week

C:\Users\IFahreza\AppData\Local\Temp/ipykernel_16656/2710808558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MA_df['MA Anomali']= MA_df['Unit']+' '+MA_df['Reported Fault/Job Description']+' ('+MA_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"


fleet desc
CAT 18M                    0.8727
Cat 16M                    0.8597
Cat 740                    0.8218
Cat 834                    0.7218
Cat D10R/T                 0.8586
Cat D8R                    0.8845
Compactor                  0.6377
Hitachi EX5500             0.7080
KOM D85ESS                 0.8770
Komatsu PC300              0.6024
Medium Truck               0.7960
TJ RENT Komatsu D65P-12    0.8580
TJ RENT Komatsu D85ESS     0.7738
TJ Rental PC200            0.8132
Volvo A40 Thiess           0.6642
Water Truck                0.8121
Name: MA, dtype: float64

In [199]:
'''MA Report Template'''

MA_report = []

    
    
for i in red_MA_this_week.index:
    Scheduled_maintenance = []
    Unschedule_maintenance = []
    
    for x in MA_df[((MA_df.Activity =='08-Planned Maintenance (PMD)')|(MA_df.Activity ==',08-Planned Maintenance (PMD)'))&(MA_df['fleet desc'] ==i)]['MA Anomali']:
        Scheduled_maintenance.append(x)
    
    for x in MA_df[((MA_df.Activity =='09-Unplanned Maintenance (UMD)')|(MA_df.Activity ==',09-Unplanned Maintenance (UMD)'))&(MA_df['fleet desc'] ==i)]['MA Anomali']:
        Unschedule_maintenance.append(x)
    
    structured_MA = {
    'Fleet':i,
    'MA':red_MA_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'UMD total hours':All_KPI.groupby('fleet desc').sum()['UMD\n(09)'][i],
    'PMD total hours':All_KPI.groupby('fleet desc').sum()['PMD\n(08)'][i],
    'PDAM total hours':All_KPI.groupby('fleet desc').sum()['PDAM\n08020\n09020'][i],
    'Scheduled maintenance' : Scheduled_maintenance,
    'Unscheduled maintenance' : Unschedule_maintenance
    
}
    MA_report.append(structured_MA)

MA_report

[{'Fleet': 'CAT 18M',
  'MA': 0.8727,
  'Total Unit': 6,
  'UMD total hours': 83.74999999999999,
  'PMD total hours': 62.5,
  'PDAM total hours': 22.33333,
  'Scheduled maintenance': ['GR1398 Schedule replace artic hitch, planned 120 hrs (144 Hours)',
   'GR1398 Electrical inspection (144 Hours)',
   'GR1398 PM 250HRS Service, Planned 6 hrs  (29.75 Hours)',
   'GR1399 Daily inspection, Planned 10 minutes (0.166666666666667 Hours)',
   'GR1399 PM 250HRS service ( Duration 4 hrs ) and backlog repair,planned 36 hrs (6 Hours)',
   'GR1399 Backlog repair (6 Hours)',
   'GR1405 Backlog repair (13.8333333333333 Hours)',
   'GR1405 Electrical Inspection (13.8333333333333 Hours)'],
  'Unscheduled maintenance': ['GR1398 Artic pin loose (29.75 Hours)',
   'GR1398 Over run : Continue backlog repair (29.75 Hours)',
   'GR1398 Glass cabin broken (144 Hours)',
   'GR1397 Secondary steering motor failure (12 Hours)',
   'GR1405 Cabin glass operator loose  (12 Hours)',
   'GR1406 Side shift cylinder lo